In [14]:
import pandas as pd
import re
from nltk import ngrams
from difflib import get_close_matches as gcm

In [2]:
df_ds = pd.read_csv('indeed-insights/data_scientist.csv')
df_ds.head()

,title,company,country,date_posted,description,title_keywords,indeed_skills,skills,no_skills
0,Data Scientist,Mission Consultancy Services Malaysia Sdn Bhd,Malaysia,2020-07-26,Identify valuable data sources and automate co...,NaN,"'C', 'C++', 'Excel', 'Hadoop', 'Java', 'R', 'S...","['Business', 'Analytical', 'Communication', 'D...",27
1,Data Scientist,AirAsia,Malaysia,2020-08-15,Job DescriptionOverviewThis role will responsi...,NaN,"'C', 'R'","['Business', 'Accuracy', 'Processing', 'Produc...",21
2,Data Scientist,CareerMaster Sdn Bhd,Malaysia,2020-07-26,Position to be based in Kuala Lumpur.We are re...,NaN,"'C', 'GIS', 'Go', 'Python', 'R', 'SQL', 'Stati...","['Business', 'Computer Science', 'Microsoft', ...",30
3,Data Scientist Executive,GENO Management,Malaysia,2020-08-18,Position : Data Scientists ExecutiveLocation :...,NaN,"'AWS', 'C', 'Go', 'Python', 'R', 'Regression',...","['Business', 'Data Science', 'Analytical', 'Bu...",39
4,Data Scientist,PLUS SOLAR SYSTEMS SDN BHD,Malaysia,2020-07-26,Work With Stakeholders Throughout The Organiza...,NaN,'C',"['Business', 'Accuracy', 'Development', 'Marke...",9


In [16]:
skills = pd.read_csv('skills_db2/skill.csv')
skills.head()

,skill_id,skill
0,1,Applied Science
1,2,Arts and Humanities
2,3,Business
3,4,Computer Science
4,5,Data Science


In [25]:
def print_job(df, i):
    row = df.loc[i]
    print('Title:', row['title'])
    print('Description:', row['description'])
    print('Indeed Skills:', row['indeed_skills'])
    
def test_extract(df, i):
    row = df.loc[i]
    info = row['title'] + ' ' + row['description']
    skills = extract_skills(info)
    skills.sort()
    print(skills)

def extract_skills(info):
    # Remove ordered list with alphabets: a), b), c),...
    words = re.sub(r'[\s\t\n]+[a-zA-Z\s*]\)+', ' ', info)
    words = re.sub('[\n|,|.|/|\(|\)]', ' ', words).lower().split()
    bigrams = [' '.join(g) for g in ngrams(words, 2)]
    trigrams = [' '.join(g) for g in ngrams(words, 3)]
    results = []
    for skill in skills['skill'].unique().tolist():
        s = skill.lower()
        if '(' in s:
            abb = s[s.find("(")+1:s.find(")")]
            if abb in info:
                continue
            s = re.sub(r"[\(].*?[\)]", "", s)
        s2 = s.split()
        if len(s2) == 1:
            if len(gcm(s, words, cutoff=0.9)) > 0:
                results.append(skill)
        elif len(s2) == 2:
            if len(gcm(s, bigrams, cutoff=0.9)) > 0:
                results.append(skill)
        elif len(s2) == 3:
            if len(gcm(s, trigrams, cutoff=0.85)) > 0:
                results.append(skill)
        else:
            if len(gcm(s, trigrams, cutoff=0.8)) > 0:
                results.append(skill)
    return results

In [24]:
print_job(df_ds, 0)

Title: Data Scientist
Description: Identify valuable data sources and automate collection processesUndertake pre-processing of structured and unstructured dataAnalyze large amount of information to discover trends and patternsBuild predictive models, graph analytics, machine-learning and deep-learning algorithmsCombine models through ensemble modelingPresent information using data visualization techniquesPropose solutions and strategies to business challengesCollaborate with engineering and product development teamsRequirementsProven experience as a Data Scientist or Data AnalystExperience in data miningUnderstanding of machine-learning, advances analytics and operations researchKnowledge of R, SQL, and Pghyton; familiarity with Big Data, Java, Spark , or C++ is an assetExperience using business intelligence tools (e.g Tableau) and data frameworks (e.g Hadoop)Analytical mind and business acumenStrong math skills (e.g statistics, algebra, ) Problem solving attitudeExcellence communicati

In [26]:
test_extract(df_ds, 0)

['Algebra', 'Analytical', 'Big Data', 'Business', 'Business Intelligence', 'Business Intelligence Testing', 'Business Intelligence Tool', 'C++', 'Communication', 'Data', 'Data Visualization', 'Development', 'Engineering', 'Ensemble', 'Graph', 'Hadoop', 'Java', 'Operations', 'Presentation', 'Problem Solving', 'Product', 'Product Development', 'R', 'Spark', 'Statistics', 'Tableau', 'Visualization']
